In [1]:
using ClimaCore
using ClimaCore: Spaces, Geometry, Fields
ENV["TEST_NAME"]="sphere/nonhydrostatic_gravity_wave";

In [2]:
include("driver.jl")

[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Running sphere/nonhydrostatic_gravity_wave
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving prognostic variables to JLD2 file on day 0
[ Info: Saving progno

114.702762 seconds (78.90 M allocations: 4.436 GiB, 1.27% gc time, 50.15% compilation time)
elapsed time (ns): 114702761899
gc time (ns):      1458133657
bytes allocated:   4763404864
pool allocs:       78864899
non-pool GC allocs:37576
malloc() calls:    667
realloc() calls:   18
GC pauses:         14


In [9]:
using ClimaCoreMakie
using GLMakie, WGLMakie

In [10]:
function zonal_velocity(time_index, lev=5, sol=sol)
    zv = @. Geometry.UVVector(sol.u[time_index].c.uₕ).components.data.:1
    return ClimaCore.Spaces.level(zv,lev)
end
function meridional_velocity(time_index, lev=5, sol=sol)
    mv = @. Geometry.UVVector(sol.u[time_index].c.uₕ).components.data.:2
    return ClimaCore.Spaces.level(zv,lev)
end
function vertical_velocity(time_index, lev=5, sol=sol)
    ᶠw = @. Geometry.WVector(sol.u[time_index].f.w).components.data.:1
    ᶜw = @. ᶜinterp(ᶠw)
    return ClimaCore.Spaces.level(ᶜw,lev)
end
function temperature_perturbation(time_index, lev=5, sol=sol)
     Y = sol.u[time_index]
    ᶜρ = Y.c.ρ
    ᶜu = Geometry.UVVector.(Y.c.uₕ).components.data.:1
    ᶜv = Geometry.UVVector.(Y.c.uₕ).components.data.:2
    ᶠw = Geometry.WVector.(Y.f.w).components.data.:1
    ᶜw = ᶜinterp.(ᶠw)
    ᶜuvw = @. Geometry.UVWVector(Y.c.uₕ) + Geometry.UVWVector(ᶜinterp(Y.f.w))
    ᶜz = ᶜlocal_geometry.coordinates.z
    eint = @. Y.c.ρe / ᶜρ - (grav * ᶜz) - 1/2 * norm_sqr(ᶜuvw)
    T = @. eint / cv_d + T_tri 
    return ClimaCore.Fields.level(T .- 300,lev)
end
ind_endtime = length(sol)
@info "Number of timesteps available is $(ind_endtime))"

[ Info: Number of timesteps available is 193)


In [12]:
maximum(temperature_perturbation(50))

-33.51751863236166

In [13]:
# animation settings
framerate = 10
time_iterator = range(1, ind_endtime)
fig = ClimaCoreMakie.viz(zonal_velocity(1)).figure
record(fig, "zonal_velocity.mp4", time_iterator; framerate = framerate) do timeind 
    ClimaCoreMakie.viz(zonal_velocity(timeind))
end

JSServe.App(WGLMakie.var"#145#146"{Scene, Base.RefValue{Any}}(Scene (800px, 600px):
  0 Plots
  1 Child Scene:
    └ Scene (768px, 568px), Base.RefValue{Any}(nothing)))

"zonal_velocity.mp4"

In [14]:
zvfig = Figure()
zvax = Axis(zvfig[1, 1])
sl_t = Slider(zvfig[2,1], range = 1:1:ind_endtime, startvalue = 1)
sl_z = Slider(zvfig[1,2], range = 1:1:10, horizontal = false, startvalue = 1)

Slider()

In [16]:
dyn_temp = Makie.lift(sl_t.value, sl_z.value) do it, iz
    temperature_perturbation(it, iz)
end
ClimaCoreMakie.viz!(dyn_temp)

Combined{ClimaCoreMakie.viz, Tuple{ClimaCore.Fields.Field{IJFH{Float64, 5, SubArray{Float64, 4, Array{Float64, 5}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}}, true}}, ClimaCore.Spaces.SpectralElementSpace2D{ClimaCore.Topologies.Topology2D{ClimaCore.Meshes.EquiangularCubedSphere{ClimaCore.Domains.SphereDomain{Float64}, ClimaCore.Meshes.NormalizedBilinearMap}, CartesianIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, LinearIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, NamedTuple{(), Tuple{}}}, ClimaCore.Spaces.Quadratures.GLL{5}, ClimaCore.Geometry.SphericalGlobalGeometry{Float64}, IJFH{ClimaCore.Geometry.LocalGeometry{(1, 2, 3), ClimaCore.Geometry.LatLongZPoint{Float64}, Float64, StaticArrays.SMatrix{3, 3, Float64, 9}}, 5, SubArray{Float64, 4, Array{Float64, 5}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Ba

In [95]:
using ClimaCore: Geometry, Meshes, Domains, Topologies, Spaces, Operators
using NCDatasets
using ClimaCoreTempestRemap

In [152]:
datafile_cc = "nhgw_sphere.nc"
NCDataset(datafile_cc, "c") do nc
    # defines the appropriate dimensions and variables for a space coordinate
    def_space_coord(nc, hv_center_space)
    # defines the appropriate dimensions and variables for a time coordinate (by default, unlimited size)
    nc_time = def_time_coord(nc)
    cspace = hv_center_space
    # define variables
    nc_rho = defVar(nc, "rho", Float64, cspace, ("time",))
    nc_e = defVar(nc, "e", Float64, cspace, ("time",))
    nc_u = defVar(nc, "u", Float64, cspace, ("time",))
    nc_v = defVar(nc, "v", Float64, cspace, ("time",))
    nc_w = defVar(nc, "w", Float64, cspace, ("time",))
    @show nc_w
    
    # write data to netcdf file
#    for i = 1:length(sol.u)
    for i = 1:1
        Y = sol.u[i]
        nc_time[i] = sol.t[i]

        # extract fields and convert to orthogonal coordinates
        Yc = Y.c
        Yf = Y.f
        uₕ = Geometry.UVVector.(Yc.uₕ)
        w = ᶜinterp.(Geometry.WVector.(Yf.w))

        # write fields to file
        nc_rho[:,i] = Yc.ρ
        nc_e[:,i] = Yc.ρe ./ Yc.ρ
        nc_u[:,i] = uₕ.components.data.:1
        nc_v[:,i] = uₕ.components.data.:2
        nc_w[:,i] = w.components.data.:1
        @assert size(nc_rho) === size(nc_w)
    end
end
typeof(hv_center_space)
typeof(hv_face_space)

nc_w = w (2400 × 10 × 0)
  Datatype:    Float64
  Dimensions:  ncol × z × time



ClimaCore.Spaces.ExtrudedFiniteDifferenceSpace{ClimaCore.Spaces.CellFace, ClimaCore.Spaces.SpectralElementSpace2D{ClimaCore.Topologies.Topology2D{ClimaCore.Meshes.EquiangularCubedSphere{ClimaCore.Domains.SphereDomain{Float64}, ClimaCore.Meshes.NormalizedBilinearMap}, CartesianIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, LinearIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, NamedTuple{(), Tuple{}}}, ClimaCore.Spaces.Quadratures.GLL{5}, ClimaCore.Geometry.SphericalGlobalGeometry{Float64}, IJFH{ClimaCore.Geometry.LocalGeometry{(1, 2), ClimaCore.Geometry.LatLongPoint{Float64}, Float64, StaticArrays.SMatrix{2, 2, Float64, 4}}, 5, Array{Float64, 4}}, Nothing, IJFH{Float64, 5, Array{Float64, 4}}, IFH{ClimaCore.Geometry.SurfaceGeometry{Float64, ClimaCore.Geometry.AxisTensor{Float64, 1, Tuple{ClimaCore.Geometry.LocalAxis{(1, 2)}}, StaticArrays.SVector{2, Float64}}}, 5, Array{Float64, 3}}, NamedTuple{(), Tuple{}}}, ClimaCore.Topologies.I

In [170]:
# write out our cubed sphere mesh
meshfile_cc = "mesh_cubedsphere.g"
write_exodus(meshfile_cc, hv_center_space.horizontal_space.topology)

# write out RLL mesh
nlat = 60
nlon = 120
meshfile_rll = "mesh_rll.g"
rll_mesh(meshfile_rll; nlat = nlat, nlon = nlon)

# construct overlap mesh
meshfile_overlap = "mesh_overlap.g"
overlap_mesh(meshfile_overlap, meshfile_cc, meshfile_rll)

# construct remap weight file
weightfile = "remap_weights.nc"
remap_weights(
    weightfile,
    meshfile_cc,
    meshfile_rll,
    meshfile_overlap;
    in_type = "cgll",
    in_np = 5,
    verbose=true
)

# apply remap
datafile_rll = "data_rll.nc"
apply_remap(datafile_rll, 
            datafile_cc, 
            weightfile, 
            ["rho","e", "u", "v", "w"],
            verbose=true)

Parameters:
  --in_mesh <string> ["mesh_cubedsphere.g"] 
  --out_mesh <string> ["mesh_rll.g"] 
  --ov_mesh <string> ["mesh_overlap.g"] 
  --in_type <string> ["cgll"] [fv|cgll|dgll]
  --out_type <string> ["fv"] [fv|cgll|dgll]
  --out_map <string> ["remap_weights.nc"] 
  --in_meta <string> [""] 
  --out_meta <string> [""] 
  --in_concave <bool> [false] 
  --out_concave <bool> [false] 
  --in_np <integer> [5] 
  --out_np <integer> [4] 
  --method <string> [""] 
  --mono <bool> [false] 
  --nobubble <bool> [false] 
  --nocorrectareas <bool> [false] 
  --noconserve <bool> [false] 
  --nocheck <bool> [false] 
  --sparse_constraints <bool> [false] 
  --in_data <string> [""] 
  --out_data <string> [""] 
  --var <string> [""] 
  --ncol_name <string> ["ncol"] 
  --out_double <bool> [false] 
  --preserve <string> [""] 
  --preserveall <bool> [false] 
  --fillvalue <double> [0.000000] 
  --out_format <string> ["Netcdf4"] [Classic|Offset64Bits|Netcdf4|Netcdf4Classic]
-------------------------------

LoadError: failed process: Process(setenv(`[4m/Users/asridhar/.julia/artifacts/78565ae29c612203abb1c795c962d729e22e70ff/bin/ApplyOfflineMap[24m [4m--map[24m [4mremap_weights.nc[24m [4m--var[24m [4mrho,e,u,v,w[24m [4m--in_data[24m [4mnhgw_sphere.nc[24m [4m--out_data[24m [4mdata_rll.nc[24m`,["PATH=/Users/asridhar/.julia/artifacts/e39202f7052633874a5334e0802af97bdf3b2406/bin:/Users/asridhar/.julia/artifacts/78565ae29c612203abb1c795c962d729e22e70ff/bin:/Users/asridhar/Research/Codes/julia/usr/bin:/Users/asridhar/.julia/conda/3/bin:/Users/local/bin:/Users/asridhar/bin:/usr/local/opt/bison/bin:~/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin", "GKS_ENCODING=utf8", "GKS_USE_CAIRO_PNG=true", "ITERM_PROFILE=Default", "USER=asridhar", "GKSwstype=nul", "COLUMNS=80", "SHELL=/bin/bash", "COLORFGBG=15;0", "LC_TERMINAL_VERSION=3.3.9"  …  "GRDIR=/Users/asridhar/.julia/artifacts/a48fcb2f05fd2670032049a6f22c619ec559f3c4/lib/..", "DISPLAY=/private/tmp/

In [92]:
Y.c

NamedTuple{(:ρ, :ρe, :uₕ), Tuple{Float64, Float64, ClimaCore.Geometry.AxisTensor{Float64, 1, Tuple{ClimaCore.Geometry.CovariantAxis{(1, 2)}}, StaticArrays.SVector{2, Float64}}}}-valued Field:
  ρ: [1.099, 0.98199, 0.8749, 0.777068, 0.687872, 0.606722, 0.533061, 0.466365, 0.406139, 0.351913  …  1.099, 0.98199, 0.8749, 0.777068, 0.687872, 0.606722, 0.533061, 0.466365, 0.406139, 0.351913]
  ρe: [24449.4, 27664.6, 29768.6, 30931.6, 31306.5, 31030.2, 30224.9, 28998.8, 27448.1, 25657.0  …  24449.4, 27664.6, 29768.6, 30931.6, 31306.5, 31030.2, 30224.9, 28998.8, 27448.1, 25657.0]
  uₕ: 
    components: 
      data: 
        1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [172]:
Y

2-blocked 122400-element ClimaCore.Fields.FieldVector{Float64, NamedTuple{(:c, :f), Tuple{ClimaCore.Fields.Field{VIJFH{NamedTuple{(:ρ, :ρe, :uₕ), Tuple{Float64, Float64, ClimaCore.Geometry.AxisTensor{Float64, 1, Tuple{ClimaCore.Geometry.CovariantAxis{(1, 2)}}, StaticArrays.SVector{2, Float64}}}}, 5, Array{Float64, 5}}, ClimaCore.Spaces.ExtrudedFiniteDifferenceSpace{ClimaCore.Spaces.CellCenter, ClimaCore.Spaces.SpectralElementSpace2D{ClimaCore.Topologies.Topology2D{ClimaCore.Meshes.EquiangularCubedSphere{ClimaCore.Domains.SphereDomain{Float64}, ClimaCore.Meshes.NormalizedBilinearMap}, CartesianIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, LinearIndices{3, Tuple{Base.OneTo{Int64}, Base.OneTo{Int64}, Base.OneTo{Int64}}}, NamedTuple{(), Tuple{}}}, ClimaCore.Spaces.Quadratures.GLL{5}, ClimaCore.Geometry.SphericalGlobalGeometry{Float64}, IJFH{ClimaCore.Geometry.LocalGeometry{(1, 2), ClimaCore.Geometry.LatLongPoint{Float64}, Float64, StaticArrays.SMatrix{2, 2, Fl